# Eda file

## 1. Data acquisition and overview

In [35]:
import pandas as pd

income_df = pd.read_csv("income_1996.csv", sep=";")
income_df.head(10)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
5,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K
6,49,Private,160187,9th,5,Married-spouse-absent,Other-service,Not-in-family,Black,Female,0,0,16,Jamaica,<=50K
7,52,Self-emp-not-inc,209642,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,45,United-States,>50K
8,31,Private,45781,Masters,14,Never-married,Prof-specialty,Not-in-family,White,Female,14084,0,50,United-States,>50K
9,42,Private,159449,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,5178,0,40,United-States,>50K


In [36]:
income_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             48842 non-null  int64 
 1   workclass       47879 non-null  object
 2   fnlwgt          48842 non-null  int64 
 3   education       48842 non-null  object
 4   education-num   48842 non-null  int64 
 5   marital-status  48842 non-null  object
 6   occupation      47876 non-null  object
 7   relationship    48842 non-null  object
 8   race            48842 non-null  object
 9   sex             48842 non-null  object
 10  capital-gain    48842 non-null  int64 
 11  capital-loss    48842 non-null  int64 
 12  hours-per-week  48842 non-null  int64 
 13  native-country  48568 non-null  object
 14  income          48842 non-null  object
dtypes: int64(6), object(9)
memory usage: 5.6+ MB


In [37]:
income_df['income'] = income_df['income'].replace({'<=50K': 0, '>50K': 1})
income_df['income'].value_counts()

C:\Users\moutt\AppData\Local\Temp\ipykernel_5800\3585989556.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  income_df['income'] = income_df['income'].replace({'<=50K': 0, '>50K': 1})


income
0    37155
1    11687
Name: count, dtype: int64

Replacing '>50K' by 1 and '<=50K' by 0 enables us to make significant calculations like correlations with other numerical variables.
Here we can see that there is a higher amount of people, more than triple the amount, having income equal to 0, so earning 50K/year or less.
So our data is disproportionate. 

In [ ]:
from scipy.stats import pointbiserialr

corr, p_value = pointbiserialr(income_df['fnlwgt'], income_df['income'])
print(f"Correlation: {corr}, p-value: {p_value}")

Correlation: -0.006338859530113218, p-value: 0.16124980594810825


Correlation is close to 0 and p-value is well over 0.05, so no correlation to our target variable 'income'. We can drop this column as it isn't correlated to the income and that we don't have any particular information on what this column refers to.

In [39]:
income_df.drop('fnlwgt', axis=1, inplace=True)

## 2. Target Variable Analysis (Check Imbalances)

In [44]:
income_percentages = income_df['income'].value_counts(normalize=True) * 100
print(income_percentages)

income
0    76.071823
1    23.928177
Name: proportion, dtype: float64


Our target column is very imbalanced as we have 76% of our dataset having an income under 50k/year and only 24% having an income over 50K/year.

Metric Choice:

- Accuracy: Is insufficient because the 76% majority class (<=50K) artificially inflates the score. 
- F1-score: It's the harmonic average of Precision and Recall. This metric provides a single, robust measure that balances the model's ability to correctly find the minority class (Recall) and avoid false alarms (Precision).
- ROC AUC (Area Under the Curve): Measures the model's overall power to rank and distinguish between the two classes. It is the best metric because it assesses performance across all decision thresholds, making it insensitive to the class imbalance itself.

## 3. Data Quality and Missing Value Strategy

In [52]:
import numpy as np

#Select all categorical columns
categorical_cols = income_df.select_dtypes(include='object').columns

#Clean the values by stripping spaces and replace '?' with NaN
income_df[categorical_cols] = income_df[categorical_cols].apply(lambda x: x.str.strip().replace('?', np.nan))

#Calculate the percentage of missing values across the whole DataFrame
missing_summary = (income_df.isnull().sum() / len(income_df)) * 100

missing_df = missing_summary[missing_summary > 0].sort_values(ascending=False)
print("Columns with Missing Data (after '?' conversion):")
print(missing_df.map('{:.2f}%'.format))

Columns with Missing Data (after '?' conversion):
occupation        5.75%
workclass         5.73%
native-country    1.75%
dtype: object


The percentage of missing value in these columns is very low. We have to decide if we want to either delete those rows or replace those rows with the most frequent value in each column.

In [53]:
#Method 1 : Imputation

#Calculate the most frequent value for the missing columns with mode()
mode_workclass = income_df['workclass'].mode()[0]
mode_occupation = income_df['occupation'].mode()[0]
mode_native = income_df['native-country'].mode()[0]

#Impute the NaNs using the calculated modes in a NEW DataFrame
income_df_imputed = income_df.copy()
income_df_imputed['workclass'].fillna(mode_workclass, inplace=True)
income_df_imputed['occupation'].fillna(mode_occupation, inplace=True)
income_df_imputed['native-country'].fillna(mode_native, inplace=True)

# 3. Check the result
print(f"Original Row Count: {len(income_df)}")
print(f"Row Count AFTER Imputation: {len(income_df_imputed)}")
print("Missing values after imputation (should be 0 for these columns):")
print(income_df_imputed[['workclass', 'occupation', 'native-country']].isnull().sum())

Original Row Count: 48842
Row Count AFTER Imputation: 48842
Missing values after imputation (should be 0 for these columns):
workclass         0
occupation        0
native-country    0
dtype: int64


C:\Users\moutt\AppData\Local\Temp\ipykernel_5800\415875024.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  income_df_imputed['workclass'].fillna(mode_workclass, inplace=True)
C:\Users\moutt\AppData\Local\Temp\ipykernel_5800\415875024.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as

In [54]:
#Method 2 : Deletion

#Delete rows on a copy of income_df
income_df_deleted = income_df.dropna(subset=['workclass', 'occupation', 'native-country'], how='any').copy()

#Number of rows lost
rows_lost = len(income_df) - len(income_df_deleted)
percentage_lost = (rows_lost / len(income_df)) * 100

print(f"Original Row Count: {len(income_df)}")
print(f"Row Count AFTER Deletion: {len(income_df_deleted)}")
print(f"Rows Lost: {rows_lost} ({percentage_lost:.2f}%)")

Original Row Count: 48842
Row Count AFTER Deletion: 45222
Rows Lost: 3620 (7.41%)


We decided to drop the following columns: workclass, occupation, and native-country, which contained missing values. These columns have a very high number of unique, non-numeric categories.

Filling missing values was not appropriate because:

- The data is highly granular, with many categories and very few repeated entries for similar rows.
- Imputing values would introduce a lot of randomness and bias, as there are no obvious options for replacement.
- Keeping these columns with arbitrary imputation could exaggerate small groups and distort the dataset.

In short, dropping these columns preserves data integrity and avoids introducing noise.

In [58]:
income_df = income_df_deleted

## 4. Detailed Feature Analysis (Categorical)